In [27]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time

class Scraper():

    def __init__(self):
        self.driver = Chrome('./chromedriver')
        url = 'https://www.zoopla.co.uk/to-rent/property/liverpool/?beds_min=1&page_size=25&price_frequency=per_month&price_max=700&view_type=list&q=Liverpool%2C%20Merseyside&radius=0&results_sort=newest_listings&search_source=refine'
        # Scraper Class to Navigate Website
        self.driver.get(url)
        self.link_list = []
        self.full_data_list = []
        self.pages = 0
        self.delay = 10

# Look for cookies button and accept cookies
    def load_and_accept_cookies(self):
        try:
            self.driver.implicitly_wait(10)
            self.driver.switch_to.frame('gdpr-consent-notice') # This is the id of the frame
            accept_cookies_button = WebDriverWait(self.driver, self.delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="save"]')))
            accept_cookies_button.click()
            time.sleep(1)
        except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
            self.driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame
            accept_cookies_button = self.driver.find_element_by_xpath('//*[@id="save"]')
            accept_cookies_button.click()
            time.sleep(1)

        except:
            pass

    def get_links(self):
        self.driver.implicitly_wait(5)
        self.prop_container = self.driver.find_element_by_xpath('//div[@class="css-1itfubx e9x22bw0"]')
        self.prop_list = self.prop_container.find_elements_by_xpath('./div')
        self.driver.implicitly_wait(5)
        for href_links in self.prop_list:
            link_tag = href_links.find_element_by_tag_name('a')
            links = link_tag.get_attribute('href')
            self.link_list.append(links)

        print(f'There are {len(self.link_list)} properties in this page')
        self.next_page()

        

 # Go to the next page   
    def next_page(self):
        self.driver.implicitly_wait(10)
        if self.pages == 0:
            next_page_button = self.driver.find_element_by_xpath('//li[@class="css-qhg1xn-PaginationItemPreviousAndNext-PaginationItemNext eaoxhri2"]')
            next_page_button.click()
            self.pages += 1
            print(self.pages)
            self.driver.implicitly_wait(10)
            cancel_button = self.driver.find_element_by_xpath('//button[@data-testid="modal-close"]')
            cancel_button.click()
            self.driver.implicitly_wait(10)
            self.get_links()

        elif self.pages == 5:
            self.driver.implicitly_wait(10)
            print(self.link_list)

        else:
            self.driver.implicitly_wait(10)
            next_page = self.driver.find_element_by_xpath('//li[@class="css-qhg1xn-PaginationItemPreviousAndNext-PaginationItemNext eaoxhri2"]')
            next_page.click()
            self.pages += 1
            print(self.pages)
            self.driver.implicitly_wait(10)
            self.get_links()
    
    def get_data(self):
        self.num_prop = len(self.link_list)
        for i in range(self.num_prop):
            self.driver.get(self.link_list[i])
            self.driver.implicitly_wait(10)
            self.data = {i: {"rent_price": [], "weekly_price": [],"bed_and_bath_num": [], "description": [], "address": [], 'image':[]}}
            # Try to find each corresponding information and append 'None' if not present
            try:
                rent_price = self.driver.find_element_by_xpath('//p[@data-testid = "price"]').text
                self.data[i]['rent_price'].append(rent_price)
            except NoSuchElementException:
                self.data[i]['rent_price'].append('None')

            try:
                weekly_price = self.driver.find_element_by_xpath('//p[@data-testid="rentalfrequency-and-floorareaunit"]').text
                self.data[i]['weekly_price'].append(weekly_price)
            except NoSuchElementException:
                self.data[i]['weekly_price'].append('None')

            try:
                bed_and_bath_num = self.driver.find_element_by_xpath('//div[@class="c-PJLV c-PJLV-iiNveLf-css"]').text
                self.data[i]['bed_and_bath_num'].append(bed_and_bath_num)
            except NoSuchElementException:
                self.data[i]['bed_and_bath_num'].append('None')

                
            try:
                description = self.driver.find_element_by_xpath('//div[@data-testid="listing_description"]').text
                self.data[i]['description'].append(description)
            except NoSuchElementException:
                self.data[i]['description'].append('None')

            try:
                address = self.driver.find_element_by_xpath('//address[@ data-testid="address-label"]').text
                self.data[i]['address'].append(address)
            except NoSuchElementException:
                self.data[i]['address'].append('None')
            try:
                image = self.driver.find_element_by_xpath('/html/body/div[2]/main/div[1]/div[1]/section/div[1]/div/div/div[1]/div/ol/li[2]/div/picture/img')
                img_src = image.get_attribute('src')
                self.data[i]['image'].append(img_src)
            except:
                self.data[i]['image'].append('None')

            print(self.data)
            self.full_data_list.append(self.data)
            self.driver.implicitly_wait(10)
        print(self.full_data_list)
           

    def scrape_website(self):
        self.load_and_accept_cookies()
        self.get_links()
        self.get_data()
        self.driver.quit()

new_Scraper_Website = Scraper()
new_Scraper_Website.scrape_website()


There are 25 properties in this page
1
There are 50 properties in this page
2
There are 75 properties in this page
3
There are 100 properties in this page
4
There are 125 properties in this page
5
There are 150 properties in this page
['https://www.zoopla.co.uk/to-rent/details/61534068/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/61533829/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/50170648/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/54928148/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/49756941/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/61532428/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-rent/details/57353011/?search_identifier=9b39c97dca058720d9bc605d74ac55ee', 'https://www.zoopla.co.uk/to-

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: Unable to receive message from renderer
  (Session info: chrome=101.0.4951.54)
